# **Summaries of each graph using Tableau**


---

### **Image 1: Demand-Side (Heat/Pressure)**

#### **Indices Tracked**:

* **Compensation Pressure Index (Blue)**: Tracks wage pressure from rising pay, wage growth, and workforce participation.
* **Labor Market Flow Index (Red)**: Measures job movement (quits, layoffs, temp hiring).
* **Labor Tightness Index (Orange)**: Reflects competition for workers and hiring confidence.

#### **Key Takeaways**:

* All three indices dip significantly during the 2008–2009 recession and again during COVID-19 (2020).
* Post-2020 shows a spike, particularly in **Labor Tightness** and **Compensation Pressure**, indicating a hot labor market.
* 2023–2025 shows some cooling, especially in job flows and tightness, but compensation pressures remain moderately elevated.
<img src="Demand-Side.png" alt="Frozen Market Index" width="900"/>

---

### **Image 2: Supply-Side (Slack/Friction)**

#### **Indices Tracked**:

* **Labor Distress Index (Red)**: High values reflect underemployment and job market congestion.
* **Hiring Friction Index (Blue)**: Captures mismatch in filling open roles.
* **Hiring Latency Index (Orange)**: Shows delays between job postings and hires.
* **Latent Labor Slack Index (Green)**: Proxy for hidden slack like part-time underemployment.

#### **Key Takeaways**:

* **Labor Distress** peaked post-2008 and briefly in 2020, consistent with downturns.
* **Friction and latency** have remained mildly elevated post-COVID, indicating slower employer responsiveness.
* **Latent slack** is relatively stable, but overall supply-side pressure eased significantly after the COVID spike.
<img src="Supply-Side.png" alt="Frozen Market Index" width="900"/>

---

### **Image 3: Composite — Frozen Market Index**

#### **Panel View**:

* Combines both **Supply-Side** and **Demand-Side** components.
* **Frozen Market Index** shown on the right plots a z-scored composite from 2006 to 2025.

#### **Key Takeaways**:

* Index spikes in 2009 and again in 2020, aligning with major labor market disruptions.
* Sustained downward trend from 2010 to 2019 indicates a more fluid and healthier labor market.
* Post-COVID freeze was short but sharp, with recovery ongoing into 2025.
* The index serves as an effective real-time signal of labor market dysfunction.
<img src="Overall.png" alt="Frozen Market Index" width="900"/>

---

### **Image 4: Frozen Market Index Only**

#### **Description**:

* Isolated view of the **Frozen Market Index** from 2006–2025.

#### **Key Takeaways**:

* Confirms the dual peaks in market freeze during the 2008 Great Recession and the 2020 COVID crisis.
* Shows recovery arcs post-recession, with 2023–2025 levels still below zero, indicating gradual return to market normalcy.
* Helpful for visualizing long-term labor market thawing after major freezes.

<img src="Frozen_Market_Index.png" alt="Frozen Market Index" width="400"/>

---

Let me know if you’d like to annotate these trends directly on the visuals or incorporate this summary into a presentation/report.


Great — you've uploaded a Jupyter notebook titled **`Demand_vs_Supply.ipynb`**, which likely includes labor market indicators and the Frozen Market Index. Here's a step-by-step outline for how to **forecast how “frozen” the job market will be in the future** using time series modeling.

---

### 🔁 Step-by-Step Forecasting Pipeline for Frozen Market Index (FMI)

---

### **Step 1: Load & Explore the Data**

* Load the relevant time series columns, particularly:

  * `observation_date`
  * `Frozen_Market_Index` or `Frozen_Market_Index_z`
  * Related indicators like `Labor_Tightness_Index`, `Labor_Distress_Index`, `Hiring_Friction_Index`, etc.

👉 Check for:

* Nulls
* Frequency (monthly? quarterly?)
* Trends & seasonality

---

### **Step 2: Preprocessing**

* Convert `observation_date` to datetime and set it as the index.
* Resample to monthly if needed (`.resample("M").mean()`).
* Handle missing values (interpolate or fill).
* Optionally smooth noisy signals with a rolling mean or z-score.

---

### **Step 3: Create Lag Features**

To capture time dependencies:

```python
for lag in range(1, 5):
    df[f'FMI_lag_{lag}'] = df['Frozen_Market_Index'].shift(lag)
```

You can also lag other indicators (e.g. `Labor_Tightness_Index_lag1`, etc.)

---

### **Step 4: Choose a Forecasting Model**

**Options:**

1. **ARIMA/SARIMA** – univariate

   * Best for baseline FMI-only forecasts
2. **VAR (Vector AutoRegression)** – multivariate

   * Use FMI + labor indicators
3. **XGBoost Regressor or RandomForestRegressor**

   * Tabular supervised regression
   * Needs lagged features as predictors
4. **Prophet (by Meta)**

   * Simple, interpretable, additive model
   * Great for seasonality & trend analysis

---

### **Step 5: Train/Test Split**

Use a **time-aware split**:

```python
train = df.loc[:'2023-12']
test = df.loc['2024-01':]
```

---

### **Step 6: Train & Predict**

For example, with **ARIMA**:

```python
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(train['Frozen_Market_Index'], order=(3,1,2))
model_fit = model.fit()
forecast = model_fit.forecast(steps=len(test))
```

Or **with XGBoost**:

```python
from xgboost import XGBRegressor

X_train = train.drop(columns=['Frozen_Market_Index'])
y_train = train['Frozen_Market_Index']
X_test = test.drop(columns=['Frozen_Market_Index'])

model = XGBRegressor()
model.fit(X_train, y_train)
forecast = model.predict(X_test)
```

---

### **Step 7: Evaluate**

Use:

* RMSE / MAE
* Visual: Plot actual vs. predicted FMI

```python
plt.plot(test.index, test['Frozen_Market_Index'], label='Actual')
plt.plot(test.index, forecast, label='Forecast')
plt.legend()
plt.show()
```

---

### **Step 8: Forecast Into the Future**

Once your model is performing well, generate future timestamps and forecast FMI into 2025+.

With ARIMA:

```python
future_forecast = model_fit.forecast(steps=12)  # 12 months ahead
```

---

### ⚙️ Optional Enhancements

* Use **SHAP** with XGBoost to interpret what indicators drive job market freezes
* Use rolling-window retraining
* Use PCA to reduce indicator dimensionality

---

Would you like me to open your notebook now and begin implementing this step-by-step in the code, or just create a fresh version of this pipeline in a new notebook?
